In [ ]:
import numpy as np
from sus.protocol_designer import System, Protocol, Potential, Compound_Protocol
from sus.protocol_designer.protocol import sequential_protocol
from IPython import display
from IPython.display import HTML
from quick_sim import setup_sim
from edward_tools.coupled_fq_potential import coupled_flux_qubit_pot, coupled_flux_qubit_force, coupled_fq_pot
import edward_tools.fq_runner as fq_runner
from edward_tools.visualization import animate_sim_flux_qubit

import kyle_tools as kt
import matplotlib.pyplot as plt

import importlib, os, hashlib, json
from edward_tools import coupled_fq_protocol_library, cfq_runner
from PARAMETER_INPUT import *
from PARAMETER_INPUT import _lambda, _theta, _eta
from ControlParameter import *

from edward_tools import coupled_fq_protocol_library
import edward_tools.cfq_batch_sweep as cfq_batch_sweep
%load_ext jupyternotify

coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system
get_potential_shot_at_different_t = coupled_fq_protocol_library.get_potential_shot_at_different_t
get_potential_shot_at_different_t_1D = coupled_fq_protocol_library.get_potential_shot_at_different_t_1D
create_simple_protocol_parameter_dict = coupled_fq_protocol_library.create_simple_protocol_parameter_dict
create_system_from_storage_and_computation_protocol = coupled_fq_protocol_library.create_system_from_storage_and_computation_protocol
coupled_fq_runner = importlib.reload(cfq_runner)
coupled_fq_protocol_library = importlib.reload(coupled_fq_protocol_library)
create_system = coupled_fq_protocol_library.create_system

get_potential_along_a_1D_cutline = coupled_fq_protocol_library.get_potential_along_a_1D_cutline
plotCutlines = coupled_fq_protocol_library.plotCutlines


In [ ]:
"""
# step 0: modify parameters
- All the parameters are stored in a separate file PARAMETER_INPUT
- You can override some of the parameters here.
"""
N = 50_00
params['N'] = N
params['dt'] = dt/10

In [ ]:
"""
# step 1: Define potential
"""
coupled_fq_default_param = [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, x_c0]
[phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound] = np.array([4, 4, 4, 4])/time_scale_factor

coupled_fq_domain = [[-phi_1_bound, -phi_2_bound, -phi_1dc_bound, -phi_2dc_bound], \
                     [phi_1_bound, phi_2_bound, phi_1dc_bound, phi_2dc_bound]]

coupled_fq_pot = Potential(coupled_flux_qubit_pot, coupled_flux_qubit_force, 14, 4,\
                           default_params = coupled_fq_default_param,  relevant_domain = coupled_fq_domain)

In [ ]:
params

In [ ]:
"""
# step 2: Define initial condition and protocol
"""
manual_domain=[np.array([-5, -5]), np.array([5, 5])]
phi_1_dcx, phi_2_dcx = 0, 0
phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx

In [ ]:
# time length
time_length_1 = 80
time_length_2 = 40
time_length_3 = 80
time_length_4 = 80
time_length_5 = 60
time_length_6 = 60

phi_1_dcx_on = 2.8
phi_2_dcx_on = 2.8
M_12_on = -0.5

In [ ]:
initial_parameter_dict = {
        "U0_1": U0_1,     "U0_2": U0_2,     "gamma_1": gamma,  "gamma_2": gamma,
        "beta_1": beta_1,   "beta_2": beta_2,   "d_beta_1": d_beta_1 ,   "d_beta_2": d_beta_2,
        "phi_1_x": 0,  "phi_2_x": 0,  "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_off,
        "M_12": M_12_off, 'x_c': x_c
}


protocol_list_options = [
    {"placeholder": ""},
    # forward
    {"duration":time_length_1, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    {"duration":time_length_2, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt"},
    {"duration":time_length_3, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(3) raise the barrier"},
    {"duration":time_length_4, "phi_1_dcx": phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(4) 4 well potential 1st time"}, 
    {"duration":time_length_5, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
    {"duration":time_length_6, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
    
    # reverse
    {"duration":time_length_6, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(-6) 4 well potential 2nd time"}, 
    {"duration":time_length_5, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(-5) anti-mix in x direction"}, 
    {"duration":time_length_4, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(-4) anti-raise the barrier 1st time"},
    {"duration":time_length_3, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12":  M_12_on,  "name":"(-3) anti-raise the barrier"},
    {"duration":time_length_2, "phi_1_dcx":phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(-2) anit-conditional tilt"},
    {"duration":time_length_1, "phi_1_dcx":phi_1_dcx_off,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(-1) anti- mix in y direction"}
]

In [ ]:
zeroDissipation = False
params['sim_params'] = [_lambda, _theta, _eta]
if zeroDissipation:
    params['sim_params'] = [_lambda * 0, _theta, _eta * 0]



"""
0 -> 1 -> 0     (0, 0, 0), (0, 3, 0), (0, 0, 0)
1 -> 2 -> 1  (0, 3, 0), (0, 3, -0.8), (0, 3, 0)
2 -> 3 -> 2  (0, 3, -0.8), (0, 0, -0.8), (0, 3, -0.8)
3 -> 4 -> 3  (0, 0, -0.8), (0, 0, 0), (0, 0, -0.8)
4 -> 5 -> 4  (0, 0, 0), (3, 0, 0), (0, 0, 0)
5 -> 6 -> 5  (3, 0, 0), (0, 0, 0), (3, 0, 0)
"""
TR_initial_condition = [
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_off), 
    (phi_1_dcx_off, phi_2_dcx_on,  M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_on), 
    (phi_1_dcx_off, phi_2_dcx_off, M_12_off), 
    (phi_1_dcx_on,  phi_2_dcx_off, M_12_off)]

protocol_index = 0

# initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index-1]
initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"] = TR_initial_condition[protocol_index]

rest = {"duration": 30, "name":"rest"}
# protocol_list = [
#     # forward
#     protocol_list_options[protocol_index + 1],
#     # protocol_list_options[1],
#     # protocol_list_options[2],
#     # protocol_list_options[3],
#     # protocol_list_options[4],
#     # protocol_list_options[5],
#     # protocol_list_options[6],    
    
#     # reverse
#     # protocol_list_options[-5],
#     # protocol_list_options[-4],
#     # protocol_list_options[-3],
#     # protocol_list_options[-2],
#     # protocol_list_options[-1]
#     protocol_list_options[-(protocol_index + 1)]
# ]

protocol_list= [
    {"duration": 100, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_off,  "name":"(1) mix in y direction"},
    {"duration": 100, "name":"rest"},
    {"duration":40, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_on,  "M_12": M_12_on,   "name":"(2) conditional tilt in y"},
    {"duration":40, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_on,   "name":"(3) raise the barrier"},
    # {"duration":100, "phi_1_dcx": phi_1_dcx_on, "phi_2_dcx": phi_2_dcx_off,  "M_12": M_12_on,   "name":"(4) conditional tilt in x"},
    {"duration":100, "phi_1_dcx": phi_1_dcx_on,  "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(5) mix in x direction"}, 
    # rest,
    {"duration":100, "phi_1_dcx": phi_1_dcx_off, "phi_2_dcx": phi_2_dcx_off, "M_12": M_12_off,  "name":"(6) 4 well potential"}, 
    
]   



print(params['sim_params'])
print(initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"])
for x in protocol_list:
    print(x)
# print(protocol_list[0]["phi_1_dcx"], protocol_list[0]["phi_2_dcx"], protocol_list[0]["M_12"], )
# print(protocol_list[1]["phi_1_dcx"], protocol_list[1]["phi_2_dcx"], protocol_list[1]["M_12"], )


In [ ]:

"""
# step 3: create the relevant storage protocol and computation protocol
"""
computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
                                                                    protocol_list)
storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

In [ ]:
"""
# step 4: create the coupled_fq_runner
"""
cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
                                                storage_protocol= storage_protocol, \
                                                computation_protocol= comp_protocol)
cfqr.initialize_sim()
cfqr.set_sim_attributes()
init_state_saved = cfqr.init_state
manual_domain=[np.array([-5, -5])/time_scale_factor, np.array([5, 5])/time_scale_factor]

# equilibrium state  analysis

In [ ]:
# from IPython.display import clear_output


# """
# 0 -> 1 -> 0     (0, 0, 0), (0, 3, 0), (0, 0, 0)
# 1 -> 2 -> 1  (0, 3, 0), (0, 3, -0.8), (0, 3, 0)
# 2 -> 3 -> 2  (0, 3, -0.8), (0, 0, -0.8), (0, 3, -0.8)
# 3 -> 4 -> 3  (0, 0, -0.8), (0, 0, 0), (0, 0, -0.8)
# 4 -> 5 -> 4  (0, 0, 0), (3, 0, 0), (0, 0, 0)
# 5 -> 6 -> 5  (3, 0, 0), (0, 0, 0), (3, 0, 0)
# """

# initial_parameter_dict["phi_1_dcx"] = phi_1_dcx_off
# initial_parameter_dict["phi_2_dcx"] = 2.5
# initial_parameter_dict["M_12"] = -0.7

# """
# # step 3: create the relevant storage protocol and computation protocol
# """
# computation_protocol_parameter_dict = coupled_fq_protocol_library.customizedProtocol(initial_parameter_dict, \
#                                                                     protocol_list)
# storage_protocol, comp_protocol = create_system(computation_protocol_parameter_dict)

# cfqr = cfq_runner.coupledFluxQubitRunner(potential = coupled_fq_pot, params = params, \
#                                                 storage_protocol= storage_protocol, \
#                                                 computation_protocol= comp_protocol)
# cfqr.initialize_sim()
# cfqr.set_sim_attributes()
# init_state_saved = cfqr.init_state
# clear_output()
# print("finish")

In [ ]:
# plt.scatter(init_state_saved[:, 0, 0], init_state_saved[:, 1, 0])
# # fig, ax = plt.subplots(1, 2, figsize = [10, 4])
# # ax[1].scatter(init_state_saved[:, 0, 0], init_state_saved[:, 1, 0])

# real simulation

In [ ]:
# step 5: perform simulations

simResult = cfq_batch_sweep.simulateSingleCoupledFluxQubit(params, initial_parameter_dict, protocol_list, \
                                        initial_state = init_state_saved, manual_domain = manual_domain, \
                                        phi_1_dcx = phi_1_dcx,  phi_2_dcx = phi_2_dcx, \
                                        percentage = 1, \
                                        as_step = np.s_[::100])

cfqr = simResult["cfqr"]

print(f"simulation_date = {simResult['simulation_data']['simulation_date']}")
print(f"simulation time = {simResult['simulation_data']['simulation_time']}s")

# animations

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 5
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)



In [ ]:
from IPython.display import HTML
from IPython import display
print(f"simulation_date = {simResult['simulation_data']['simulation_date']}")
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

# work analysis

## work distributions

In [ ]:
step_time_array = np.cumsum(np.array([x["duration"] for x in protocol_list]))/params['dt']
name_array = [x["name"] for x in protocol_list]

from scipy.stats import sem 

plt.hist(simResult["work_distribution"], bins = 95)
plt.show()
jarzyn_term = np.mean(np.exp(-simResult["work_distribution"]))

avg_W = np.mean(simResult["work_distribution"])
error_W = sem(simResult["work_distribution"])
jarzyn_term_error = np.exp(-avg_W) * error_W

print(f"simulation_date = {simResult['simulation_data']['simulation_date']}")
print(f"mean work = {avg_W:.3f} ± {error_W:.5f}")
print(f'jarzyn = {jarzyn_term:.3f} ± {jarzyn_term_error:.5f}')
print([x["duration"] for x in protocol_list])
print(time_length_1, time_length_2, time_length_3, time_length_4, time_length_5, time_length_6)

## work statistics

In [ ]:
step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
skip_step = int(len(step_array) * 0.05)
work_mean = simResult["work_statistic"][:,0]
work_std = simResult["work_statistic"][:,1]

coarse_step_array = step_array[::skip_step]
coarse_step_array = np.append(coarse_step_array, step_array[-1])
coarse_work_mean_array = work_mean[::skip_step]
coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array * 3)

for i, t in enumerate(step_time_array):
    plt.axvline(x = t, color = 'b', label = 'axvline - full height')
    # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
    plt.annotate(f"{i+1}", 
            xy=(t, 4.5),  
            # xytext=(quantile_75, 2.05*1e9),
            color='red', 
            ha='center', 
            size=16)

In [ ]:
work_mean

In [ ]:
WD_at_step_array = np.array([work_mean[i-1] for i in step_time_array.astype(int)])

In [ ]:
net_WD = WD_at_step_array[1:] - WD_at_step_array[0:-1]

In [ ]:
net_WD

In [ ]:
step_time_array.astype(int)

In [ ]:
t_array = np.cumsum([p["duration"] for p in protocol_list])

# crook analysis

## type 1

In [ ]:
counts, bin_edges = np.histogram(simResult["work_distribution"], bins = np.linspace(-2, 2, 50),density=True)
counts, bin_edges = np.histogram(simResult["work_distribution"], bins = np.linspace(-0.2, 0.2, 20),density=True)

In [ ]:
log_P_ratio = np.log(counts / counts[::-1])

In [ ]:
plt.figure(figsize=(5, 5))
plt.plot(bin_edges[1:], bin_edges[1:], "--")
plt.scatter(bin_edges[:-1], log_P_ratio)
plt.xlabel("W")
plt.ylabel("log ratio")

## type 2

In [ ]:
# crook analysis
# np.linspace(0.005, 0.2, 100)
test_w = np.linspace(-1.0, 1.0, 100)
dw = 0.001
crook_analysis_result = []

for _w in test_w:
    array = simResult["work_distribution"]

    # The 0 index is to remove the single element array
    number_of_plus_W = len(np.where(np.logical_and(array <= _w + dw, array >= _w - dw))[0])
    number_of_minus_W = len(np.where(np.logical_and(array <= -_w + dw, array >= -_w - dw))[0])
    crook_analysis_result.append(np.array([_w, number_of_plus_W, number_of_minus_W * np.exp(_w)]))

crook_analysis_result = np.array(crook_analysis_result)    
print(f"dw = {dw}, dt = {params['dt']}, N = {params['N']}")
print("LHS: number of +W, RHS = number of -W * np.exp(+W)]")
all_w = crook_analysis_result[:, 0]
all_lhs = crook_analysis_result[:, 1]
all_rhs = crook_analysis_result[:, 2]
plt.xlabel("W")
plt.plot(all_w, all_lhs, "x", label = "LHS")
plt.plot(all_w, all_rhs, "x", label = "RHS")
plt.legend()
plt.show()
# print("p(W) = p(-W) exp(W)")
# print(f"LHS = {number_of_plus_W}, RHS = {number_of_minus_W * np.exp(test_w)}")
print(f"mean work = {avg_W:.3f} and the std error of W = {error_W:.5f}")
print(f'jarzyn = {jarzyn_term:.3f}, std error= {jarzyn_term_error:.5f}')
print([x["duration"] for x in protocol_list])
print(time_length_1, time_length_2, time_length_3, time_length_4, time_length_5, time_length_6)

In [ ]:
result = {"N": params['N'], "dt": params['dt'], "W_mean": avg_W, "W_std": error_W, 
          "jarzyn": jarzyn_term, "jarzyn_error": jarzyn_term_error}

In [ ]:
jarzyn_err_analytic = [ np.exp(-s["W_mean"]) * s["W_std"] for s in data]

In [ ]:
plt.hist(jarzyn_err_analytic)

In [ ]:


# step_array = np.arange(simResult["work_statistic"][:,0].shape[0])
# skip_step = int(len(step_array) * 0.02)
# work_mean = simResult["work_statistic"][:,0]
# work_std = simResult["work_statistic"][:,1]

# coarse_step_array = step_array[::skip_step]
# coarse_step_array = np.append(coarse_step_array, step_array[-1])
# coarse_work_mean_array = work_mean[::skip_step]
# coarse_work_mean_array = np.append(coarse_work_mean_array, work_mean[-1])
# coarse_work_std_array = np.append(work_std[::skip_step], work_std[-1])

# plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)

# for i, t in enumerate(step_time_array):
#     plt.axvline(x = t, color = 'b', label = 'axvline - full height')
#     # plt.text('2017-07-02 16:30:00',0.005,'BigNews1',rotation=90,va='top')
#     plt.annotate(f"{i+1}", 
#             xy=(t, 4.5),  
#             # xytext=(quantile_75, 2.05*1e9),
#             color='red', 
#             ha='center', 
#             size=16)

# print(f'jarzyn = {jarzyn_term}')
# print([x["duration"] for x in protocol_list])

# first_half = work_mean[0: len(work_mean)//2]
# second_half = work_mean[len(work_mean)//2:][::-1]
# plt.plot(step_array[0:len(step_array)//2], np.abs(first_half - second_half))

In [ ]:
# print(initial_parameter_dict["phi_1_dcx"], initial_parameter_dict["phi_2_dcx"], initial_parameter_dict["M_12"])
# print(protocol_list)

In [ ]:
# animations
vmin, vmax = 0, 100
frame_skip = 10
# phi_1_dc, phi_2_dc = phi_1_dcx, phi_2_dcx
all_state = simResult["cfqr"].sim.output.all_state['states']
simResult["cfqr"].system.protocol_list = protocol_list
time_range = (computation_protocol_parameter_dict["t"][0], computation_protocol_parameter_dict["t"][-1])

ani,_,_ = animate_sim_flux_qubit(all_state, system = simResult["cfqr"].system ,
                                   times = time_range, frame_skip=frame_skip, color_by_state=True,
                                   vmin = vmin, vmax = vmax,
                                   manual_domain = manual_domain)

In [ ]:
# from IPython.display import HTML
from IPython import display
video = ani.to_html5_video()
html = display.HTML(video)
display.display(html)

In [ ]:
# import time
# now = str(time.time())
# sim_id = hashlib.sha256(bytes(now, encoding='utf8')).hexdigest()




# # save image and distributions
# np.save(f'cfq_data/tr_analysis/{sim_id}_work_distribution.npy', simResult["work_distribution"])
# plt.hist(simResult["work_distribution"], bins = 45)
# plt.savefig(f'cfq_data/tr_analysis/{sim_id}_work_distribution.png')
# plt.close()

# np.save(f'cfq_data/tr_analysis/{sim_id}_work_statistic.npy', simResult["work_statistic"])
# plt.errorbar(coarse_step_array, coarse_work_mean_array, yerr = coarse_work_std_array)
# plt.plot(step_array[0:len(step_array)//2], np.abs(first_half - second_half))
# plt.savefig(f'cfq_data/tr_analysis/{sim_id}_work_statistic.png')
# plt.close()

# np.save(f"cfq_data/tr_analysis/{sim_id}_state.npy", cfqr.sim.output.all_state["states"])

In [ ]:
# get_potential_shot_at_different_t_1D(cfqr, computation_protocol_parameter_dict, manual_domain = manual_domain)

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

Z_score = 2
N = 302
with open("cfq_data/variance_analysis_for_TRS_3.json") as f:
    data = json.load(f)
    W_mean_samples = np.array([d["W_mean"] for d in data])
    W_mean_err_samples = np.array([d["W_std"] for d in data])
    jarzyn_samples = np.array([d["jarzyn"] for d in data])
    jarzyn_err_samples = np.array([d["jarzyn_error"] for d in data])
    
    jarzyn_err_expt = np.std(jarzyn_samples)
    jarzyn_err_analytic = np.mean([ np.exp(-s["W_mean"]) * s["W_std"] for s in data])
    print("expt error: ", jarzyn_err_expt)
    print("analytic error: ", jarzyn_err_analytic)


#     data.append(result)
#     with open("cfq_data/variance_analysis_for_TRS_3.json", "w") as f_w:
#         json.dump(data, f_w)

In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

Z_score = 2
N = 302
with open("cfq_data/variance_analysis_for_TRS_whole_forward.json") as f:
    data = json.load(f)
    print(data)
#     W_mean_samples = np.array([d["W_mean"] for d in data])
#     W_mean_err_samples = np.array([d["W_std"] for d in data])
#     jarzyn_samples = np.array([d["jarzyn"] for d in data])
#     jarzyn_err_samples = np.array([d["jarzyn_error"] for d in data])
    
#     jarzyn_err_expt = np.std(jarzyn_samples)
#     jarzyn_err_analytic = np.mean([ np.exp(-s["W_mean"]) * s["W_std"] for s in data])
#     print("expt error: ", jarzyn_err_expt)
#     print("analytic error: ", jarzyn_err_analytic)


#     data.append(result)
#     with open("cfq_data/variance_analysis_for_TRS_3.json", "w") as f_w:
#         json.dump(data, f_w)

In [ ]:
f = data[0]["fidelity"]

In [ ]:
summaryText = ""
for x in f:
    initialLocation = x["initial"]["location"]
    initialCount = x["initial"]["count"]
    summaryText = f"initial: {initialLocation} ({initialCount}), final: "
    rightLocationCount = sum([y["count"] for y in x["final"]])
    goodRatio = rightLocationCount/initialCount * 100
    for y in x["final"]:
        summaryText += f"{y['location']} ({y['count']}/{rightLocationCount},{y['count']/rightLocationCount * 100: .3g}%),"
    summaryText += f" goodRatio:{goodRatio: .3g}%"
    # print(summaryText)


In [ ]:
summaryText